In [ ]:
!pip install praw
import praw
# from prawcore.exceptions import Forbidden
import pandas as pd
from datetime import datetime
import os   

sub_r = 'StopSelfHarm'

reddit = praw.Reddit(client_id='---------------', 
                     client_secret='----------------', 
                     user_agent='hashem',
                     check_for_async=False)

root = '/content/drive/MyDrive/reddit_data/'
path_sub = root + sub_r + '_submissions/'
path_com = root + sub_r + '_comments/'
path_out = root + 'redditors_' + sub_r + '/'
if not os.path.exists(path_out):
    os.mkdir(path_out)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 5.3 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 


In [ ]:
files_com = [x for x in next(os.walk(path_com))[2]]
authors_com = set()
for f in files_com:
    df_ = pd.read_csv(path_com + f, usecols = ['author'], lineterminator='\n', engine='c')
    authors_com.update(df_['author'])

files_sub = [x for x in next(os.walk(path_sub))[2]]
authors_sub = set()
for f in files_sub:
    df_ = pd.read_csv(path_sub + f, usecols = ['author'], lineterminator='\n', engine='c')
    authors_sub.update(df_['author'])

unique_authors = authors_sub.copy()
unique_authors.update(authors_com)

In [ ]:
len(authors_sub), len(authors_com), len(unique_authors)

(4218, 8541, 10276)

In [ ]:
# find already searched authors
author_filter = ['AutoModerator', '[deleted]', '[removed]']
previous_authors_s = set(author_filter)
previous_authors_c = set(author_filter)

previous_label_c = -1
previous_label_s = -1

def get_num(string):
    num = ''
    for c in string:
        if c.isdigit():
          num += c
    return int(num)

if os.listdir(path_out):
    files = next(os.walk(path_out))[2]
    for f in files:
        df_ = pd.read_csv(path_out + f, lineterminator='\n')
        num = get_num(f)

        if 'submissions' in f:
            previous_authors_s.update(df_['author'])
            if num > previous_label_s:
                previous_label_s = num

        elif 'comments' in f:
            previous_authors_c.update(df_['c_author'])
            if num > previous_label_c:
                previous_label_c = num

In [ ]:
previous_label_s, previous_label_c, len(previous_authors_s), len(previous_authors_c)

In [ ]:
submissions_all = []
comments_all = []

threshold = 5000
batch_s = previous_label_s + 1
batch_c = previous_label_c + 1

for author in unique_authors:
    submissions = []
    comments = []
    
    if not (author in previous_authors_c):
      try:
          for c in reddit.redditor(author).comments.new(limit=1000):
              comments.append(c)
      except:
          continue

    if not (author in previous_authors_s):
      try:
          for s in reddit.redditor(author).submissions.new(limit=1000):
              submissions.append(s)
      except:
          continue
        
    submissions_all.extend(submissions)
    comments_all.extend(comments)
    
    if len(submissions_all) > threshold:
        posts_l = []
        
        for p in submissions_all:
            posts_l.append([datetime.utcfromtimestamp(p.created).strftime('%Y-%m-%d'),
                            datetime.utcfromtimestamp(p.created).strftime('%H:%M:%S'),
                            p.subreddit_name_prefixed, 
                            p.subreddit_id, 
                            p.subreddit_subscribers, 
                            p.id, 
                            p.author, 
                            p.author_flair_text, 
                            p.name, 
                            p.title, 
                            p.selftext, 
                            p.edited, 
                            p.is_reddit_media_domain, 
                            p.is_original_content, 
                            p.is_meta,p.is_self, 
                            p.score, 
                            p.ups, 
                            p.downs, 
                            p.upvote_ratio, 
                            p.total_awards_received, 
                            p.num_comments, 
                            p.num_crossposts, 
                            p.stickied, 
                            p.allow_live_comments, 
                            p.url, 
                            p.num_reports, 
                            p.send_replies, 
                            p.over_18,
                            p.user_reports, 
                            p.distinguished, 
                            p.gilded, 
                            p.likes, 
                            p.view_count, 
                            p.visited,
                            p.spoiler, 
                            p.is_video, 
                            p.is_crosspostable, 
                            p.is_created_from_ads_ui, 
                            p.hidden,
                            p.hide_score, 
                            p.content_categories, 
                            p.contest_mode, 
                            p.category,
                            p.author_is_blocked,
                            p.no_follow,
                            p.top_awarded_type,
                            p.locked,
                            p.mod_reason_title,
                            p.mod_reports])

        posts=pd.DataFrame(posts_l,columns=['date', 
                                            'time',
                                            'subreddit', 
                                            'subreddit_id', 
                                            'subreddit_subscribers', 
                                            'submission_id', 
                                            'author', 
                                            'author_flair', 
                                            'name', 
                                            'text', 
                                            'selftext', 
                                            'time_edited', 
                                            'media', 
                                            'is_original_content', 
                                            'is_meta', 
                                            'is_self', 
                                            'score', 
                                            'ups', 
                                            'downs', 
                                            'upvote_ratio', 
                                            'n_awards', 
                                            'n_comments', 
                                            'n_crossposts', 
                                            'stickied', 
                                            'allow_live_comments', 
                                            'url', 
                                            'n_reports', 
                                            'send_replies',
                                            'over_18', 
                                            'user_reports', 
                                            'distinguished', 
                                            'gilded', 
                                            'likes', 
                                            'view_count',
                                            'visited', 
                                            'spoiler', 
                                            'is_video', 
                                            'is_crosspostable', 
                                            'is_created_from_ads_ui', 
                                            'hidden', 
                                            'hide_score', 
                                            'content_categories', 
                                            'contest_mode', 
                                            'category',
                                            'author_blocked',
                                            'no_follow',
                                            'top_awarded_type',
                                            'locked',
                                            'mod_reason_title',
                                            'mod_reports'])

        posts.to_csv(path_out + 'redditors_in_' + sub_r + '_elsewhere_submissions_{}.csv'.format(batch_s))
        
        batch_s += 1
        submissions_all = []
        
    
    if len(comments_all) > threshold:
        comments_l = []
        
        for c in comments_all:
            comments_l.append([datetime.utcfromtimestamp(c.created).strftime('%Y-%m-%d'),
                               datetime.utcfromtimestamp(c.created).strftime('%H:%M:%S'),
                               c.subreddit_name_prefixed, 
                               c.subreddit_id, 
                               c._submission, 
                               c.parent_id, 
                               c.id, 
                               c.link_id,
                               c.author, 
                               c.author_flair_text, 
                               c.name, 
                               c.is_submitter, 
                               c.body, 
                               c.permalink,
                               c.edited, 
                               c.score, 
                               c.ups, 
                               c.downs,
                               c.score_hidden,
                               c.controversiality,
                               c.total_awards_received, 
                               c.comment_type, 
                               c.send_replies, 
                               c.unrepliable_reason, 
                               c.user_reports, 
                               c.stickied, 
                               c.distinguished, 
                               c.gilded, 
                               c.no_follow, 
                               c.author_is_blocked, 
                               c.likes,
                               c.collapsed,
                               c.collapsed_reason,
                               c.locked,
                               c.mod_reason_title,
                               c.mod_reports,
                               c.num_reports,
                               c.top_awarded_type])

        comments = pd.DataFrame(comments_l, columns=['date', 
                                                     'time',
                                                     'subreddit', 
                                                     'subreddit_id', 
                                                     'submission_id', 
                                                     'parent_id', 
                                                     'comment_id', 
                                                     'link_id',
                                                     'c_author', 
                                                     'c_author_flair', 
                                                     'c_name', 
                                                     'is_submitter', 
                                                     'text', 
                                                     'url',
                                                     'time_edited', 
                                                     'score', 
                                                     'ups', 
                                                     'downs', 
                                                     'score_hidden',
                                                     'controversiality', 
                                                     'awards', 
                                                     'comment_type',
                                                     'send_replies', 
                                                     'unrepliable_reason', 
                                                     'user_reports', 
                                                     'stickied', 
                                                     'distinguished', 
                                                     'gilded', 
                                                     'no_follow', 
                                                     'c_author_blocked', 
                                                     'likes',
                                                     'collapsed',
                                                     'collapsed_reason',
                                                     'locked',
                                                     'mod_reason_title',
                                                     'mod_reports',
                                                     'n_reports',
                                                     'top_awarded_type'])      

        comments.to_csv(path_out + 'redditors_in_' + sub_r + '_elsewhere_comments_{}.csv'.format(batch_c))
        
        batch_c += 1
        comments_all = []




### last remaining
posts_l = []
for p in submissions_all:
    posts_l.append([datetime.utcfromtimestamp(p.created).strftime('%Y-%m-%d'),
                    datetime.utcfromtimestamp(p.created).strftime('%H:%M:%S'),
                    p.subreddit_name_prefixed, 
                    p.subreddit_id, 
                    p.subreddit_subscribers, 
                    p.id, 
                    p.author, 
                    p.author_flair_text, 
                    p.name, 
                    p.title, 
                    p.selftext, 
                    p.edited, 
                    p.is_reddit_media_domain, 
                    p.is_original_content, 
                    p.is_meta,p.is_self, 
                    p.score, 
                    p.ups, 
                    p.downs, 
                    p.upvote_ratio, 
                    p.total_awards_received, 
                    p.num_comments, 
                    p.num_crossposts, 
                    p.stickied, 
                    p.allow_live_comments, 
                    p.url, 
                    p.num_reports, 
                    p.send_replies, 
                    p.over_18,
                    p.user_reports, 
                    p.distinguished, 
                    p.gilded, 
                    p.likes, 
                    p.view_count, 
                    p.visited,
                    p.spoiler, 
                    p.is_video, 
                    p.is_crosspostable, 
                    p.is_created_from_ads_ui, 
                    p.hidden,
                    p.hide_score, 
                    p.content_categories, 
                    p.contest_mode, 
                    p.category,
                    p.author_is_blocked,
                    p.no_follow,
                    p.top_awarded_type,
                    p.locked,
                    p.mod_reason_title,
                    p.mod_reports])

posts=pd.DataFrame(posts_l,columns=['date', 
                                    'time',
                                    'subreddit', 
                                    'subreddit_id', 
                                    'subreddit_subscribers', 
                                    'submission_id', 
                                    'author', 
                                    'author_flair', 
                                    'name', 
                                    'text', 
                                    'selftext', 
                                    'time_edited', 
                                    'media', 
                                    'is_original_content', 
                                    'is_meta', 
                                    'is_self', 
                                    'score', 
                                    'ups', 
                                    'downs', 
                                    'upvote_ratio', 
                                    'n_awards', 
                                    'n_comments', 
                                    'n_crossposts', 
                                    'stickied', 
                                    'allow_live_comments', 
                                    'url', 
                                    'n_reports', 
                                    'send_replies',
                                    'over_18', 
                                    'user_reports', 
                                    'distinguished', 
                                    'gilded', 
                                    'likes', 
                                    'view_count',
                                    'visited', 
                                    'spoiler', 
                                    'is_video', 
                                    'is_crosspostable', 
                                    'is_created_from_ads_ui', 
                                    'hidden', 
                                    'hide_score', 
                                    'content_categories', 
                                    'contest_mode', 
                                    'category',
                                    'author_blocked',
                                    'no_follow',
                                    'top_awarded_type',
                                    'locked',
                                    'mod_reason_title',
                                    'mod_reports'])

posts.to_csv(path_out + 'redditors_in_' + sub_r + '_elsewhere_submissions_{}.csv'.format(batch_s))

comments_l = []
for c in comments_all:
    comments_l.append([datetime.utcfromtimestamp(c.created).strftime('%Y-%m-%d'),
                        datetime.utcfromtimestamp(c.created).strftime('%H:%M:%S'),
                        c.subreddit_name_prefixed, 
                        c.subreddit_id, 
                        c._submission, 
                        c.parent_id, 
                        c.id, 
                        c.link_id,
                        c.author, 
                        c.author_flair_text, 
                        c.name, 
                        c.is_submitter, 
                        c.body, 
                        c.permalink,
                        c.edited, 
                        c.score, 
                        c.ups, 
                        c.downs,
                        c.score_hidden,
                        c.controversiality,
                        c.total_awards_received, 
                        c.comment_type, 
                        c.send_replies, 
                        c.unrepliable_reason, 
                        c.user_reports, 
                        c.stickied, 
                        c.distinguished, 
                        c.gilded, 
                        c.no_follow, 
                        c.author_is_blocked, 
                        c.likes,
                        c.collapsed,
                        c.collapsed_reason,
                        c.locked,
                        c.mod_reason_title,
                        c.mod_reports,
                        c.num_reports,
                        c.top_awarded_type])

comments =pd.DataFrame(comments_l, columns=['date', 
                                            'time',
                                            'subreddit', 
                                            'subreddit_id', 
                                            'submission_id', 
                                            'parent_id', 
                                            'comment_id', 
                                            'link_id',
                                            'c_author', 
                                            'c_author_flair', 
                                            'c_name', 
                                            'is_submitter', 
                                            'text', 
                                            'url',
                                            'time_edited', 
                                            'score', 
                                            'ups', 
                                            'downs', 
                                            'score_hidden',
                                            'controversiality', 
                                            'awards', 
                                            'comment_type',
                                            'send_replies', 
                                            'unrepliable_reason', 
                                            'user_reports', 
                                            'stickied', 
                                            'distinguished', 
                                            'gilded', 
                                            'no_follow', 
                                            'c_author_blocked', 
                                            'likes',
                                            'collapsed',
                                            'collapsed_reason',
                                            'locked',
                                            'mod_reason_title',
                                            'mod_reports',
                                            'n_reports',
                                            'top_awarded_type'])      

comments.to_csv(path_out + 'redditors_in_' + sub_r + '_elsewhere_comments_{}.csv'.format(batch_c))